In [1]:
import scipy
import scipy.io
import numpy as np
from scipy.linalg import norm
from scipy.sparse import csr_matrix
from scipy.linalg import norm
import pickle
import time
from collections import defaultdict
import json
from sklearn.datasets import load_svmlight_file
import matplotlib.pyplot as plt
import pandas as pd
import numpy.matlib

$$\min_{x \in \mathbb R^n_+; \|x\|_1 \le M} f(x) = \sum_{i = 1}^N w_i^T x - \sum_{i = 1}^N y_i \ln (w_i^T x) + \lambda \|x\|_1$$

$$\min_{x \in \mathbb R^n_+; \sum_{i}x_i\le M} f(x) = \sum_{i = 1}^N w_i^T x - \sum_{i = 1}^N y_i \ln (w_i^T x) + \lambda e^Tx$$

In [3]:
def poisson(W, y, lam, x):
    """
        W -- object matrix (N x n)
        y -- labels (N)
        x -- weights (n)
        lam -- regularization param
    """ 
    dot_product = W @ x # N
#     print(x)
#     print(dot_product, end='\n\n')np.where(
    fst_term = np.sum(dot_product)
    snd_term = y.dot(np.log(dot_product))
    return fst_term - snd_term + lam * sum(x), dot_product

$$\nabla f(x) = \begin{bmatrix}\sum_{i=1}^N w_i - \sum_{i = 1}^N y_i \frac{w_i}{w_i^T x}+\lambda e\end{bmatrix}$$

In [4]:
# def grad_poisson_kamil(W, y, x, lam):
#     Btm = W.dot(x)  # Btm(i) = a_i^T x
#     Par = 1 - y / Btm
#     i = list(range(N))
#     DPar = scipy.sparse.csr_matrix((Par, (i, i)), shape=(N, N))
#     ResM = DPar.dot(W)
#     l1_grad = np.array([1 if c >=0 else -1 for c in x])
#     Grad = np.array(np.sum(ResM, axis=0)).squeeze() + lam * l1_grad
#     return Grad, Btm

In [5]:
def grad_poisson(W, y, lam, x, dot_product=None):
    """
        W -- object matrix (N x n)
        y -- labels (N)
        x -- weights (n)
        lam -- regularization param
        btm -- W @ x (N)
    """
    if dot_product is None:
        dot_product = np.squeeze(W @ x) # N
    mult=(1-(y / dot_product))
    x_term = (W.T @ mult) # n
    return x_term.T+lam



$$\nabla^2 f(x) = \sum_{i = 1}^N y_i \frac{w_i w_i^T}{(w_i^T x)^2}$$

$$(s-x)^T\nabla^2 f(x)(s-x) = \sum_{i = 1}^N y_i \frac{((s-x)^Tw_i)^2}{(w_i^T x)^2}$$

In [30]:
# def hess_poisson_kamil(W, y, lam, Btmv):
#     Rat = y / (Btmv)**2
#     i = range(N)
#     DRat = scipy.sparse.csr_matrix((Rat, (i, i)), shape=(N, N))
#     Res1 = (W.T.dot(DRat)).dot(W)
#     I = np.eye(n)
#     Res2 = lam * I
#     H = Res1 + Res2
#     # Cholesky method for inverse
# #     try:
# #         L1 = np.linalg.cholesky(H)
# #         Lc = L1.T
# #     except np.linalg.LinAlgError as err:
# #         print('Cholesky decomposition of H is failed! Regularize it ...')
# #         mineig = min(np.linalg.eig(H)[0])
# #         Lc = np.linalg.cholesky(H + (-mineig + 1e-12) * I).T
# #     iL = np.linalg.solve(Lc, I)
# #     Hinv = iL.dot(iL.T)
# #     H = H.A
#     return H

In [7]:
def hess_poisson(W, y, x,lam, Btm):
    """
        W -- object matrix (N x n)
        y -- labels (N)
        x -- weights (n)
    """
    denom = 1 / (Btm) # N
    snd_einsum = np.multiply(W, denom.reshape(-1, 1))
    fst_einsum = y.reshape(-1, 1) * snd_einsum
    return np.einsum('ij,ik->jk', fst_einsum, snd_einsum)

def hess_mult_vec(W, y, s, Btm):
    """
        W -- object matrix (N x n)
        y -- labels (N)
        x -- weights (n)
    """
    return (((W @ s)*y)/Btm).dot(W)


def hess_mult(W, y, x, Btm):
    """
        W -- object matrix (N x n)
        y -- labels (N)
        x -- weights (n)
    """
    num=y.dot((W @ x)**2/Btm)
    return num


In [ ]:
def proj_simplex_full(grad,M,lam):
    n=len(grad)
    s = np.zeros(n)
    i_max = np.argmax(-(grad))
    if grad[i_max]*M+lam*M<0:
        s[i_max] = -np.sign(grad[i_max])*M # 1 x n
    return s    